In [1]:
import pandas as pd
import numpy as np
import re

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
real = pd.read_excel('data/data.xlsx')
# Меняем названия на более приемлемые:

target_shops = [
    'АО "ТОРГОВЫЙ ДОМ "ПЕРЕКРЕСТОК"', 'ООО "АГРОТОРГ"', 'ООО "ЛЕНТА"',
   'АО "ДИКСИ ЮГ"', 'ООО "АШАН"', 'АО "ТАНДЕР"', 'ООО "БИЛЛА"', 'ООО "О`КЕЙ"',
    'ООО "АГРОАСПЕКТ"', 'ООО "АТАК"'
]

shops_rename = {
    'АО "ТОРГОВЫЙ ДОМ "ПЕРЕКРЕСТОК"': 'ПЕРЕКРЕСТОК',
    'ООО "АГРОТОРГ"': 'ПЯТЕРОЧКА',
    'ООО "ЛЕНТА"': 'ЛЕНТА',
    'АО "ДИКСИ ЮГ"': 'ДИКСИ',
    'ООО "АШАН"': 'АШАН',
    'АО "ТАНДЕР"': 'МАГНИТ',
    'ООО "БИЛЛА"': 'БИЛЛА',
    'ООО "О`КЕЙ"': 'О`КЕЙ',
    'ООО "АГРОАСПЕКТ"': 'ПЯТЕРОЧКА',
    'ООО "АТАК"': 'АТАК'
}

real = real[real['shop_name'].isin(target_shops)].copy()
real['shop_name'] = real['shop_name'].apply(lambda x: shops_rename[x])
real = real.drop_duplicates(subset=['name'])
real.sample(2)

,shop_name,name,quantity,price,sum
27350,ПЯТЕРОЧКА,3931719 H.KID Гель д/подмыв.мал.400мл,1.0,134.99,134.99
15132,ПЕРЕКРЕСТОК,4:3492564 Колбаски МИРАТОРГ 300г,1.0,160.37,160.37


In [3]:
real.head()

,shop_name,name,quantity,price,sum
19,ДИКСИ,МИНТАЙ ФИЛЕ Б/К СВ/МОР П/ПАК 8,1.0,199.90,199.90
20,ДИКСИ,БЗМЖ МОЛОКО СТРАНА ВАСИЛЬКИ У/,1.0,49.99,49.99
21,ДИКСИ,СУШКИ ТАРАЛЛИНИ С ЧЕСНОКОМ 180,1.0,29.99,29.99
22,ДИКСИ,ЯЙЦО КУРИНОЕ СТОЛОВОЕ 1КАТЕГОР,1.0,63.99,63.99
23,ДИКСИ,ПЕЧЕНЬЕ ЮБИЛЕЙНОЕ ВИТАМИНИЗИРО,2.0,29.99,59.98


In [4]:
clean = pd.read_csv('clean_data/all_clean.csv')

In [6]:
tmp = pd.read_csv('benchmarks/standard.csv')

In [7]:
def find_weight_or_volume(name: str)-> str:
    """Находит вес или объём товара."""

    pattern1 = r'\d+((\.|\,|x|X|х|Х)?)\d*([а-я]|[А-Я])*$' # ищет вес в конце
    pattern2 = r'\d+\w+' #  ищет вес в середине
    pattern3 = r'(\d+ кг)|(\d+ г)'  # находит `1 кг` или `1 г`
    result = re.search(pattern1, name)
    if result:
        return result.group(0)
    result = re.search(pattern2, name)
    if result:
        return result.group(0)
    result = re.search(pattern3, name)
    return result.group(0) if result else ''

In [6]:
# tmp['Название'].apply(find_weight_or_volume)

In [7]:
tmp['Название'] = tmp['Название'].str.lower()

In [243]:
from product_dict import PRODUCT_DICT

class NormalizeNames:
    def __init__(self, name: pd.Series):
        self.df = pd.DataFrame(name.copy()).rename(columns={'name': 'name_norm'})
        self.df['product_norm'] = None
        self.df['brand_norm'] = None
        
        self.slash_brands = {
            'deluxe/': 'deluxe',
            'nemoloko/': 'nemoloko',
            'huggies/': 'huggies',
            'figaro/': 'figaro',
            'кп/': 'красная птица',
            'кд/': 'каждый день',
            'heinz/' : 'heinz',
            'lay`s/': 'lays',
            'моя цена/': 'моя цена',
            'магнит/': 'магнит',
            'rich/': 'rich',
            '/овощная семейка': 'овощная семейка'
        }

        self.slash_product = {
            'б/йог': 'биойогурт',
            'сух/завтрак': 'сухой завтрак',
            '/пучок': 'пучок',
            'з/щ': 'зубная щётка',
            'йогурт/смусси': 'йогурт',
            'т/бумага': 'туалетная бумага',
            'укроп/петрушк': 'зелень',
            'б/полотенца': 'бумажные полотенца',
            '/сыр': 'сыр',
            'з/паста': 'зубная паста',
            'т/мыло': 'мыло',
            'ж/мыло': 'жидкое мыло',
            'СУХАРИ/': 'сухари/',
            'бум/салф': 'бумажные полотенца'
            
        }
        self.dot_brands = {
            'д.в д': 'домик в деревне',
            'хл.сп': 'хлебный спас',
            'к.ц': 'красная цена',
            'кр.ц': 'красная цена',
            'кр.цена': 'красная цена',
            'пр!ст': 'просто',
            'п.св': 'первая свежесть',
            'раст': 'растишка',
            'гранд дуэт': 'grand duet',
            'прост': 'простоквашино',
            'волнист': 'волнистые',
            'mil gen': 'miller genuine',
            'шарл': 'шарлиз',
            'домашняя кухн': 'домашняя кухня',
            'крикет': 'cricket',
            'домик в дерев': 'домик в деревне',
            'акт': 'актвия',
            'озор.пчел': 'озорная пчелка',
            'вор.сух': 'воронцовские сухарики',
            'dr.oetker': 'dr.oetker',
            'б.ю.александров': 'б. ю. александров',
            'mr.ricco': 'mr. ricco',
            'dr.korner': 'dr. korner',
            'alp.gold': 'alpen gold',
            'mil.gen.dr' : 'miller',
            'с.прид': 'сады придонья',
            'g.gold': 'gourmet gold',
            'colg.з/п': 'colgate',
            'с.пудовъ': 'с. пудовъ',
            'сибирск.коллекция': 'сибирская коллекция',
            '(данон)': 'danone',
            'lay`': 'lays',
            'кока-кола': 'coca-cola',
            'марс': 'mars',
            'нестле': 'nestle',
            'брест-литовск': 'брест-литовский',
            "lay's": 'lays',
            'хенкель': 'henkel',
            'gl.vil': 'global village',
            'bar': 'barilla',
            'tol': 'tolli',
            'юбил.': 'юбилейное',
            'астор': 'astoria',
            'бар.': 'барни',
            'рот фронт': 'ротфронт',
            'роллт': 'роллтон',
            'верхов.': 'верховье',
            'мист.': 'мистраль',
            'иммун': 'иммунеле',
            'рест.': 'рестория',
            'зд.мен.': 'здоровое меню',
            ' вв ': 'вкусвилл',
            'зелен. линия': 'зеленая линия',
            'комм.': 'коммунарка',
            'волоконовско': 'волоконовское',
            'олвейс': 'always',
            'палмолив': 'palmolive',
            'жатецкий гусь': 'zatecky gus',
            'советские традиц': 'советские традиции',
            'президент': 'president',
            'стародв': 'стародворские',
            'альметте': 'almette',
            'дом в дер': 'домик в деревне',
            'дом дер': 'домик в деревне',
            'виола виттр': 'виола виттрока',
            'золот ларец': 'золотой ларец',
            'клинск': 'клинский',
            'добр': 'добрый',
            'рублев': 'рублевский',
            'кремлев': 'кремлевский',
            'гессер': 'gosser',
            'воронц': 'воронцовские',
            'гранола': 'granola',
            'r.sp.': 'ritter sport',
            'sиб.кол.': 'сибирская коллекция',
            'яр.бройл.': 'ярославский бройлер',
            'кузм.': 'кузмино',
            'виола': 'viola',
            'чупа чупс': 'chupa chups',
            'наше солнышк': 'наше солнышко',
            'ролл': 'роллтон',
            'зева': 'zewa',
            'б.ю.алекс.': 'б.ю.александров',
            'мамба': 'mamba',
            'эком.': 'ecomilk',
            'савушк.': 'савушкин хуторок',
            'милка': 'milka',
            'виолетта': 'Violette',
            'марк.пер.': 'маркет перекресток',
            'национ.': 'националь',
            'пр.кав.': 'предгорье кавказа',
            'велк.': 'велком',
            'байс.': 'baisad',
        }
        self.brands = ['магнит', 'foxlite', 'пятерочка', 'домик в деревне', 'перекресток', 'карусель', 'лента', 'я самая',
                      'billa', 'мягкий знак', 'окей', 'саф-момент', 'рот-фронт', 'му-у', 'm&ms', 'брест-литовский', 'oral-b',
                      'mccormick', 'хлебцы-молодцы', 'coca-cola', 'агро-альянс', 'eco-botanica', 'агуша', 'фрутоняня', 'heinz',
                       'мираторг', 'вкусвилл', 'danone', 'bonduelle', 'hochland', 'greenfield', 'nescafe', 'president', 'бабаевский',
                      'доширак', 'lorenz naturals', 'colgate', 'barilla', 'tolli', 'ролтон', 'юбилейное', 'astoria', 'fa', 'из углича',
                      'моя цена', 'коммунарка', 'хлебозавод', 'орловский', 'страна васильки', 'дикси', 'первым делом', 'мера вкуса', 'щелковский',
                      'кириешки', 'кузя', 'крестьянское', 'яков давыдов', 'волковская пивоварня', 'дмитровский', 'российский',
                      'завтрак+', 'первак', 'океан', 'маракас', 'круглый год', 'фруто-няня', 'зеленая линия', 'овсяная изюминка', 'петр I', 'наше солнышко']
        
    @staticmethod
    def remove_all_words_which_contains_number(name, product, brand):
        # Todo: Создать словарь с брендами, у которых есть цифры, например, 365 ДНЕЙ, 7 DAYS
        brands_with_numeric = [
            '365 дней', '7 days', '6 соток', '5 морей', '4 сезона', '7up', '3 glocken', '1toy', '48 копеек',
            'j7', 'got2b', 'хлебозавод 28', '4life','4 life' '36 копеек', 'байкал', '101 зерно', '5 океанов', 'j-7'
        ]
        
        for br in brands_with_numeric:
            if br in name:
                brand = br
                name = name.replace(br, '')
                break
        name = ' '.join(re.sub(r'\w*\d\w*', '', word).strip() for word in name.split())
        return pd.Series([name, product, brand])
    
    def remove_all_words_which_contains_slash(self, name, product, brand):
        for key, value in self.slash_brands.items():
            if key in name:
                brand = value
                break
        
        for key, value in self.slash_product.items():
            if key in name:
                product = value
                break
        name = name.replace('кп/', '').replace('кд/', '')
        name = ' '.join(re.sub(r'\w*/\w*', '', word).strip() for word in name.split())
        return pd.Series([name, product, brand])
    
    def remove_punctuation_symbols(self, name, product, brand):
        for key, value in self.dot_brands.items():
            if key in name:
                brand = value
                name = name.replace(key, '')
                break
        
        tmp = [re.sub(r'([.,!?%:*\(|\)])', ' ', word).strip() for word in name.split()]
        name = ' '.join(word[:-1] if word.endswith('-') else word for word in tmp)
        return pd.Series([name, product, brand])
    
    @staticmethod
    def remove_one_and_two_chars(name: str):
#         whitelist = ['с', 'в', 'и', 'из']
        whitelist = []
        return ' '.join(x.strip() for x in name.split() if len(x) > 2 or x in whitelist)
#         return ' '.join(re.sub(r'\b\w{,2}\b', ' ', word).strip() for word in name.split())
    
    
    def find_brands(self, name, product, brand):
        brands_v2 = {
            'greenf': 'greenfield',
            'hochl': 'hochland',
            'nesc': 'nescafe',
            'ч лин': 'чистая линия',
            'pres': 'president',
            'бабаев': 'бабаевский',
            'дош': 'доширак',
            'бр-лит': 'брест-литовский',
            'lor': 'lorenz naturals',##### ?
            'dan': 'danone',
            'bond': 'bonduelle',
            'colg': 'colgate'
        }
        for br in self.brands:
            if br in name:
                brand = br
                name = name.replace(br, '')
                break
    
        for key, value in brands_v2.items():
            if key in name:
                brand = value
                name = name.replace(key, '')
                break
        return pd.Series([name, product, brand])
    
    @staticmethod
    def remove_words_in_blacklist(name: str):
        blacklist = ['бзмж', 'вес', 'для', 'нарезной', 'пшен', 'тенд', 'белый', 'premium', 'декор', 'ранний',
                    'струч', 'короткоплодные', 'венские', 'хлебопек', 'репчатый', 'упак', 'мытая',
                    'дуболистный', 'красный', 'газовая', 'средний', 'слив', 'перс', 'груш', 'классич', 'прод',
                     'раств', 'зерно', 'ябл-бан', 'выгодно', 'красные', 'цветы', 'раст', 'кета', 'косметик',
                    'короткоплодн', 'абр', 'жел', 'шейкер', 'гречн', 'сдоб', 'н-р', '-сл', '[м]', 'вбд',
                     'россия', 'по-домашнему', 'по-корейски', 'киш-миш', 'тд-холдинг', 'дой-пак', 'фрито',
                    'кисло-сладкий', 'рж-пш', 'оквашино', 'нар', 'фас', 'изд', 'охл', 'бел', 'дет', 'паст', 'one',
                    'сливочный', 'зеленью', 'плавл', 'ассорти', 'классическое', 'суп&соус', 'чизбургер', 'ломтик', 'чизбург',
                    'пец-хаас', 'кукурузный', 'ванильный', 'пищевой', 'баварская', 'мульт', 'крупные', 'оливлайн',
                     'черные', 'val', 'плав', 'ломт', 'асс', 'фин', 'избр', 'плавленый', 'финский', 'творожный',
                    'рисунком', 'арт', 'р-р', 'гладкие', 'крас', 'премиум', 'лбп', 'пачка', 'сушеные', 'хлебзернопрод', 'азербайджан', 'розовые',
                    'смо', 'изо', 'ивиа', 'прозр', 'барное', 'светлое', 'темное', 'ультр', 'среднеплодные', 'гладк', 'прополис', 'чесноком', 'карам',
                    'детский', 'яблоко-вишня', 'ишка', 'фасованные', 'детское', 'ржаной', 'украинск', 'оливковый', 'инд', 'окваш', 'моц', 'столовое', 'бокс',
                    'молочный', 'нарез', 'столовая', 'весенние', 'зимой', 'зер', 'зерненый', 'бород', 'белокочанная', 'ранняя', 'домашние', 'морская', 'мелкая', 'рус', 'кривское',
                    'классический', 'ригли', 'вал', 'дрож', 'ябл-черная', 'соев', 'ван', 'москов', 'докторская', 'сух', 'дрожжевое', 
                     'слоеное', 'кокосом', 'зеленый', 'спм', 'вареная', 'выпечки', 'курин', 'пшеничная', 'мытый', 'берканские', 'сладкие',
                    'навелин', 'круглый', 'пинк леди', 'желтый', 'вильямс', 'топл', 'белые', 'спз', 'кдв', 'цыпл-брой', 'сетке',
                     'длинноплодный', 'соленые', 'ский', 'бок', 'тер', 'овские', 'атл', 'маринован', 'гов', 'бородинский', 'рожки', 'пинк', 'хлебоп', 'адриатика',]
        return ' '.join(word for word in name.split() if word.strip() not in blacklist)
    
    @staticmethod
    def replace_with_product_dict(name: str):
        return ' '.join(PRODUCT_DICT.get(word.strip(), word) for word in name.split())
    
    @staticmethod
    def replace_most_commen_reductions(name: str):
        reductions = {
            'ср-во': 'средство',
            'филе-кусок': 'филе',
            'нап': 'напиток',
            'крем-мыло': 'мыло',
            'мини-сушки': 'сушки',
            'лук-репка': 'лук',
            'отбел': 'отбеливатель',
            'биотвор': 'биотворог',
            'вин': 'вино'
        }
        return ' '.join(reductions.get(word.strip(), word) for word in name.split())
        
    def find_scratch_card(self, name, product, brand):
        if 'скретч-карта' in name:
            return pd.Series(['', 'скретч-карта', 'перекресток'])
        return pd.Series([name, product, brand])
    
    @staticmethod
    def find_product(name, product):
        if not product:
            if len(name.split()) == 1:
                if name == 'мол':
                    return pd.Series(['', 'молоко'])
                if name == 'биойог':
                    return pd.Series(['', 'биойогурт'])
                product = name
                return pd.Series(['', product])
            return pd.Series([name, None]) 
        return pd.Series([name, product])

    @staticmethod
    def find_and_remove_all_english_words(name, brand):
        eng_brands = re.findall(r'\b([a-z]+)\b', name)
        if eng_brands and not brand:
            result = []
            for word in eng_brands:
                name = name.replace(word, '').replace('  ', ' ').strip()
                result.append(word)

            brand = ' '.join(result)
            if 'epica' in brand:
                brand = 'epica'
            return pd.Series([name, brand])

        # If we have brand, we should delete english words in product name
        name = re.sub(r'\b([a-z]+)\b', '', name).replace('  ', ' ').strip()
        return pd.Series([name, brand])
    
    def normalize(self):
        self.df['name_norm'] = self.df['name_norm'].str.lower()
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.remove_all_words_which_contains_number(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.remove_all_words_which_contains_slash(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df['name_norm'] = self.df['name_norm'].apply(self.replace_with_product_dict)
        
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.remove_punctuation_symbols(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.find_brands(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df['name_norm'] = self.df['name_norm'].apply(self.remove_one_and_two_chars)
        self.df['name_norm'] = self.df['name_norm'].apply(self.remove_words_in_blacklist)
        self.df['name_norm'] = self.df['name_norm'].apply(self.replace_most_commen_reductions)
        self.df[['name_norm', 'product_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.find_scratch_card(
                x['name_norm'], x['product_norm'], x['brand_norm']
            ), axis=1
        )
        self.df[['name_norm', 'brand_norm']] = self.df.apply(
            lambda x: self.find_and_remove_all_english_words(x['name_norm'], x['brand_norm']), axis=1
        )
        self.df[['name_norm', 'product_norm']] = self.df.apply(
            lambda x: self.find_product(x['name_norm'], x['product_norm']), axis=1
        )
        return self.df

In [236]:
n = NormalizeNames(real['name'])
tmp = n.normalize()
# t.head()

In [237]:
df = pd.concat([real, tmp], axis=1)[['shop_name', 'name', 'name_norm', 'product_norm', 'brand_norm']]

In [238]:
df.brand_norm.dropna().shape # 11966

(12273,)

In [239]:
df.product_norm.dropna().shape # 6741

(7167,)

## TODO:
Все данные достаточно хорошо очищены, теперь используя датасет:
1. сначла ищем бренд - нашли удаляем его
2. ищем продукты

In [16]:
df[df['name'].str.find('КОММ') != -1]

,shop_name,name,name_norm,product_norm,brand_norm
1004,ДИКСИ,ШОКОЛАД КОММУНАРКА С ПЮРЕ ИЗ К,шоколад коммунарка пюре,None,None
1006,ДИКСИ,ШОКОЛАД КОММУНАРКА С ПЮРЕ ИЗ А,шоколад коммунарка пюре,None,None
5934,МАГНИТ,КОММУНАРКА Шоколад молоч солен карамель 200г к/у,коммунарка шоколад молоч солен карамель,None,None
8768,МАГНИТ,КОММУНАРКА Шоколад гор трюф элит 200,коммунарка шоколад гор трюфель,None,None
11274,ПЕРЕКРЕСТОК,*4002898 КОММ.Конфеты БЕЛОВЕЖ.ПУЩА 1кг,комм конфеты беловеж пуща,None,None
11885,ПЕРЕКРЕСТОК,3614049 КОММУН.Какао-порошок 150г,коммун какао-порошок,None,None
17759,ПЕРЕКРЕСТОК,*3608921 КОММ.Шок.трюф.элит 200г,комм шок трюф элит,None,None
18559,ПЕРЕКРЕСТОК,3*: 3642219 КОММ.Шок.ТРЮФ.ЭЛИТ мол.нач.2,комм шок трюф элит мол нач,None,None
18560,ПЕРЕКРЕСТОК,4*: 3617637 КОММ.Шок.МОЛ.нач.вк.кап.эл.2,комм шок мол нач кап,None,None
22526,ПЕРЕКРЕСТОК,*3614049 КОММУН.Какао-порошок 150г,коммун какао-порошок,None,None


In [48]:
'lepetitmarseillais'.isalpha()

True

In [36]:
clean['Бренд'].dropna()

0        le petit marseillais
1                         axe
2                        dove
3                     schauma
4                     pantene
                 ...         
71633                   paula
71634                   paula
71635                   paula
71636                 merries
71637                 merries
Name: Бренд, Length: 71575, dtype: object

In [58]:
for i in list(range(0, 10)):
    print(f'"{i}"', end = ', ')

"0", "1", "2", "3", "4", "5", "6", "7", "8", "9", 

In [99]:
brands = clean['Бренд'].dropna()

In [143]:
rus_brands = rus_brands.apply(lambda name: ' '.join(x.strip() for x in name.split() if len(x) > 2))

In [101]:
brands = brands.drop_duplicates()

In [102]:
brands = brands.apply(lambda name: re.sub(r'\b([a-z]+)\b', '', name).replace('  ', ' ').strip())

In [106]:
brands = brands[brands != '']

In [107]:
rus_brands = brands

In [145]:
rus_brands.to_csv('clean_data/rus_brands.csv', index=False)

In [136]:
rus_brands = rus_brands[rus_brands != ".'"]

In [139]:
rus_brands = rus_brands.apply(lambda x: x.replace('.', ' ').replace('  ', ' ').strip())

In [147]:
df.sample()

,shop_name,name,name_norm,product_norm,brand_norm
12346,ПЕРЕКРЕСТОК,2:3967644 Напиток JACOBS CHOCO 4в1 12г,,напиток,jacobs choco


In [148]:
set(rus_brands) & set('jacobs choco'.split())

set()

In [157]:
clean[clean['Бренд'] == 'страна васильки']

,Название,Категория,Бренд,Вес,Продукт


In [240]:
def foo(name, brand):
    global rus_brands
    if brand or name == '':
        return pd.Series([name, brand])
        
    for word in rus_brands:
        if word in name:
            new_name = name.replace(word, '').replace('  ', ' ').strip()
            return pd.Series([new_name, word])
    return pd.Series([name, brand])
    

In [207]:
df.loc[73639:73640].apply(lambda x: foo(x['name_norm'], x['brand_norm']), axis=1)

,0,1
73639,лаваш армянский бездрожжевой,None
73640,лаваш грузинский элазян,None


In [209]:
df.loc[73639:73640]

,shop_name,name,name_norm,product_norm,brand_norm
73639,О`КЕЙ,Лаваш армянский бездрожжевой 3,лаваш армянский бездрожжевой,None,None
73640,О`КЕЙ,Лаваш грузинский 300г Элазян,лаваш грузинский элазян,None,None


In [241]:
df[['name_norm', 'brand_norm']] = df.apply(lambda x: foo(x['name_norm'], x['brand_norm']), axis=1)

In [242]:
df['brand_norm'].dropna().shape

(17541,)

In [244]:
df['product_norm'].dropna().shape

(7167,)

In [248]:
df[df['name_norm'].apply(lambda x: len(x.split()) == 1)]

,shop_name,name,name_norm,product_norm,brand_norm
27,ДИКСИ,МАЙОНЕЗ РЯБА ОЛИВКОВЫЙ 67% Д/П,майонез,None,ряба
49,ДИКСИ,БЗМЖ ПУДИНГ ГРАНД ДЕСЕРТ БЕЛ.Ш,пудинг,None,гранд десерт
83,ДИКСИ,БЗМЖ СЛИВКИ БОЛЬШАЯ КРУЖКА У/П,сливки,None,большая кружка
94,ДИКСИ,БЗМЖ СЛИВКИ БОЛЬШАЯ КРУЖКА 10%,сливки,None,большая кружка
121,ДИКСИ,БАТОН НАРЕЗНОЙ МОСКОВСКИЙ 1С Н,батон,None,московский
...,...,...,...,...,...
73896,О`КЕЙ,БЗМЖ Творожок Савушкин кокос/м,творожок,None,савушкин
73954,О`КЕЙ,Батон Аладушкин 400 г в/с,батон,None,аладушкин
73957,О`КЕЙ,БЗМЖ Кефир Пискаревский 1% 1000г,кефир,None,пискаревский
74015,О`КЕЙ,БЗМЖ Сырок Б.Ю. Александров Ка,сырок,None,александров


In [ ]:
RICH/

In [234]:
df[df['brand_norm'].apply(lambda x: not bool(x))].sample(10)

,shop_name,name,name_norm,product_norm,brand_norm
26000,ПЯТЕРОЧКА,*94430 БАЙС.Изд.мак.СПИР.в/с450г,байс мак спир,None,None
46538,АШАН,Д/С ДЕЛ РУБЕЦ 150Г,дел рубец,None,None
4359,ДИКСИ,"БЗМЖ МОЛОКО СГУЩ.РОГАЧ.7,5% С/",молоко сгущ рогач,None,None
36580,ПЯТЕРОЧКА,3351540 ВЕЛК.Сосис.ТЕЛЯЧ.c сыр.в/у 530г,велк сосис теляч сыр,None,None
45496,АШАН,ВАТРУШКА С ТВОРОГ1ШТ,,ватрушка,None
22142,ПЕРЕКРЕСТОК,15:3640864 Изюм светлый 233г,изюм светлый,None,None
41539,АТАК,МИНИ-СУШКИ СЕМ.150ГР,сушки сем,None,None
9385,МАГНИТ,НЕВСКИЙ КОНДИТЕР Конфеты волш ламп,невский кондитер конфеты волш ламп,None,None
44156,АШАН,БА Ш-НЬ ЗАЩИЩАЮЩ ЦВЕТ 100МЛ,ш-нь защищающ цвет,None,None
11865,ПЕРЕКРЕСТОК,20:3639749 Бананы фасованные 1кг,,бананы,None


In [ ]:
class BrandsFinder:
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        
    
        
        
    def find_all(self):
        pass

In [ ]:
'knorr', 

In [243]:
df[df['name_norm'].apply(lambda x: 'knorr' in x.split())]

,shop_name,name,name_norm,product_norm,brand_norm
7938,МАГНИТ,KNORR НА ВТОРОЕ для плова 27г (Юнилевер) :21,knorr второе плова юнилевер,None,None
10887,ПЕРЕКРЕСТОК,*3391162 Смесь KNORR НА ВТОРОЕ 27г,смесь knorr второе,None,None
19509,ПЕРЕКРЕСТОК,21: 3610322 Смесь сухая KNORR НА ВТОРОЕ 28г,смесь сухая knorr второе,None,None
19510,ПЕРЕКРЕСТОК,22: 3385940 Смесь KNORR НА ВТОРОЕ 32г,смесь knorr второе,None,None
21405,ПЕРЕКРЕСТОК,*3434946 Смесь KNORR НА ВТОРОЕ сухая 23г,смесь knorr второе сухая,None,None
21778,ПЕРЕКРЕСТОК,11:3190239 Суп KNORR ЧАШКА 16г,суп knorr чашка,None,None
23283,ПЕРЕКРЕСТОК,15: 3676549 Смесь KNORR НА ВТОРОЕ сухая 30г,смесь knorr второе сухая,None,None
62744,ЛЕНТА,Смесь KNORR Густой томат. по-итальянс51г,смесь knorr густой томат,None,None
62745,ЛЕНТА,Смесь KNORR Аром. Сырный по-француз.48г,смесь knorr аром сырный по-француз,None,None
62746,ЛЕНТА,Смесь KNORR Аром. Грибной по-француз.49г,смесь knorr аром грибной по-француз,None,None


In [234]:
C = Counter()
def find_all_english_words(name):
    global C
    
    C[' '.join(word for word in re.findall(r'\b([a-z]+)\b', name))] += 1

In [235]:
re.findall(r'\b([a-z]+)\b' ,'пив напиток garage lingo')

['garage', 'lingo']

In [236]:
df['name_norm'].apply(find_all_english_words);

In [304]:
clean[clean['Бренд'] == 'помидорка']

,Название,Категория,Бренд,Вес,Продукт
225,"паста томатная помидорка (десан), 270 г",Другое,помидорка,270,паста томатная
334,"томатная паста помидорка, 270г",Другое,помидорка,270г,томатная паста
492,томатная паста помидорка,"Соусы, орехи, консервы",помидорка,70 г,томатная паста
2547,томаты помидорка очищенные в собственном соку,"Соусы, орехи, консервы",помидорка,800 г,томаты
8930,лечо помидорка натуральное,"Соусы, орехи, консервы",помидорка,680 г,лечо
8953,томаты помидорка черри маринованные,"Соусы, орехи, консервы",помидорка,720 г,томаты
18700,паста томатная помидорка 70г,"Соусы, орехи, консервы",помидорка,"0,07кг.",паста томатная
24863,"томатная паста помидорка, 380г","Соусы, орехи, консервы",помидорка,"0,38кг.",томатная паста
26255,паста томатная помидорка 140г,"Соусы, орехи, консервы",помидорка,"0,14кг.",паста томатная
27424,"томатная паста помидорка, 250мл с/б","Соусы, орехи, консервы",помидорка,"0,25кг.",томатная паста


In [121]:
df[df['name_norm'].apply(lambda x: len(x.split()) == 1)]

,shop_name,name,name_norm,product_norm,brand_norm
24,ДИКСИ,ЛУК РЕПЧАТЫЙ ВЕС,лук,лук,None
28,ДИКСИ,ОГУРЦЫ 600Г,огурцы,огурцы,None
34,ДИКСИ,ПЕРЕЦ КРАСНЫЙ ВЕС,перец,перец,None
36,ДИКСИ,ТОМАТЫ ВЕС,томаты,томаты,None
45,ДИКСИ,БЗМЖ СМЕТАНА ПРОСТОКВАШИНО 15%,сметана,сметана,простоквашино
...,...,...,...,...,...
73866,О`КЕЙ,БЗМЖ Пюре ФрутоНяня яблоко/мал,пюре,пюре,фрутоняня
73883,О`КЕЙ,Вишня ОКЕЙ б/к 300г,вишня,вишня,окей
73898,О`КЕЙ,Апельсины д/сока фас кг,апельсины,апельсины,None
73911,О`КЕЙ,Виноград белый б/к 500г упак,виноград,виноград,None


(32091, 5)

# TODO через str.find


марм - мармелад
йог - йогурт
биойог - биойогурт
игр-карт: карточная игра Тролли от петёрочки
Скретч-карта THE VOICE - скретч-карта от Перекрётска 
зажиг: зажигалка
твор: творог ## если нет сырок
пив: пиво
нап: напиток
мак изделия: макароны

In [ ]:
'СОСИСК.ВЕНСК.МГА470Г'

In [469]:
'МАК.ИЗДЕЛИЯ'.lower()

'мак.изделия'

In [190]:
df[df['name_norm'].apply(lambda x: 'vil' in x.split())]

,shop_name,name,name_norm,product_norm,brand_norm
13014,ПЕРЕКРЕСТОК,3255142 GL.VIL.Маслины б/к 425г,vil маслины,None,None
14224,ПЕРЕКРЕСТОК,3633056 GL.VIL.Огурцы соленые в/у 400г,vil огурцы соленые,None,None
14837,ПЕРЕКРЕСТОК,*3369226 GL.VIL.Хрен СТОЛОВЫЙ 150г,vil хрен столовый,None,None
16299,ПЕРЕКРЕСТОК,"3630205 GL.VIL.Нектар вишневый 0,2л",vil нектар вишневый,None,None
16332,ПЕРЕКРЕСТОК,3327870 GL.VIL.Огурцы мар.целые 720мл,vil огурцы мар целые,None,None
16338,ПЕРЕКРЕСТОК,"3630206 GL.VIL.Нект.М/ФР.из см.фрук0,2л",vil нект из,None,None
16339,ПЕРЕКРЕСТОК,"3630207 GL.VIL.Нектар персик.с мяк.0,2л",vil нектар персик с мяк,None,None
16425,ПЕРЕКРЕСТОК,"3694329 GL.VIL.Сок томат.мяк/сол.0,95л",vil сок томат,None,None
17749,ПЕРЕКРЕСТОК,"3634675 GL.VIL.Нектар апельсин.0,95л",vil нектар апельсин,None,None
17802,ПЕРЕКРЕСТОК,*3631927 GL.VIL.Капуста кваш.по-дер.900г,vil капуста кваш по-дер,None,None


In [83]:
df[df['name'].str.find('ЖЕВ/РЕЗ') != -1]

,shop_name,name,name_norm,product_norm,brand_norm
50751,БИЛЛА,ЖЕВ/РЕЗ/14Г/,,None,None


In [ ]:

'кр.цена': 'красная цена',
'пр!ст': 'просто',

In [ ]:
З/ПАСТА
    HEINZ/ХАЙНЦ
    Т/бумага
    Т/мыло
    Ж/мыло
    КП/МОЛОКО
    LAY`S/ЛЕЙЗ

In [ ]:
GREENF.Чай 'HOCHL.Сыр',  NESC.Кофе' Ч.ЛИН PRES.Сыр БАБАЕВ ДОШ.Лапша БР-ЛИТ

In [485]:
'СИБИРСК.КОЛЛЕКЦИЯ'.lower()

'сибирск.коллекция'

In [57]:
C.most_common(100)

[('с', 1978),
 ('сыр', 1314),
 ('в', 872),
 ('хлеб', 716),
 ('творог', 665),
 ('молоко', 555),
 ('мол', 518),
 ('из', 508),
 ('напиток', 485),
 ('масло', 455),
 ('шоколад', 455),
 ('йогурт', 428),
 ('пиво', 411),
 ('шок', 409),
 ('сметана', 394),
 ('вода', 345),
 ('томаты', 335),
 ('яйцо', 330),
 ('чай', 313),
 ('батон', 306),
 ('колбаса', 305),
 ('филе', 303),
 ('печенье', 303),
 ('соус', 301),
 ('сок', 297),
 ('яблоки', 293),
 ('чипсы', 287),
 ('огурцы', 286),
 ('корм', 275),
 ('конфеты', 268),
 ('нап', 249),
 ('и', 247),
 ('салат', 243),
 ('вино', 240),
 ('кофе', 238),
 ('сырок', 220),
 ('агуша', 215),
 ('куриное', 203),
 ('оквашино', 202),
 ('лук', 201),
 ('пюре', 200),
 ('кефир', 200),
 ('капуста', 195),
 ('охл', 193),
 ('смесь', 190),
 ('салфетки', 189),
 ('средство', 189),
 ('сосиски', 188),
 ('нар', 179),
 ('мука', 171),
 ('дом', 167),
 ('черри', 165),
 ('паста', 164),
 ('картофель', 164),
 ('майонез', 153),
 ('хлебцы', 152),
 ('мыло', 150),
 ('паст', 149),
 ('дет', 149),
 ('he

In [53]:
from collections import Counter

C = Counter()
def lol(name):
    global C
    for word in name.split():
#         if word.isalpha():
#             continue
        C[word] += 1
        
df['name_norm'].apply(lol);

In [ ]:
# find:
мол шок игр лента красный нап куриное дет кур бел  твор упак карта дск дом стол смесь цена короткоплодные скретч gold паст изд колб перекресток зеленый домик бумага йог мытая прод микс печ дней

# blacklist
фас охл пятерочка нар репчатый

In [39]:
l = 'мол шок игр лента красный нап куриное дет кур бел шокол много  твор упак карта дск дом стол смесь цена короткоплодные скретч gold паст изд колб перекресток зеленый домик бумага йог мытая прод микс печ дней'.split()

In [90]:
real[real['name_new'].str.find(i) > -1].sample(30)

(579, 6)

In [ ]:
Д.В Д - домик в деревне
К.Ц. - красная цена
ПР!СТ - просто
подгуз-трус - подгузники
365 ДНЕЙ 

In [264]:
real[real['name_new'] == 'лента']

,shop_name,name,quantity,price,sum,name_new,lower_name
47279,АШАН,ДВ.ВСП.ЛЕНТА 24ММХ2М,1.0,45.50,45.50,лента,дв.всп.лента 24ммх2м
59449,ЛЕНТА,Б/полотенца ЛЕНТА 2-сл.4шт,1.0,89.99,89.99,лента,б/полотенца лента 2-сл.4шт
60251,ЛЕНТА,Б/полотенца ЛЕНТА 2х сл 1шт,1.0,59.99,59.99,лента,б/полотенца лента 2х сл 1шт
60252,ЛЕНТА,Б/полотенца ЛЕНТА 2х сл 1шт,1.0,59.99,59.99,лента,б/полотенца лента 2х сл 1шт
60253,ЛЕНТА,Б/полотенца ЛЕНТА 2х сл 1шт,1.0,59.99,59.99,лента,б/полотенца лента 2х сл 1шт
60254,ЛЕНТА,Б/полотенца ЛЕНТА 2х сл 1шт,1.0,59.99,59.99,лента,б/полотенца лента 2х сл 1шт
61454,ЛЕНТА,Б/полотенца ЛЕНТА 2х сл 1шт,1.0,59.99,59.99,лента,б/полотенца лента 2х сл 1шт
61605,ЛЕНТА,Б/полотенца ЛЕНТА 2х сл 1шт,1.0,59.99,59.99,лента,б/полотенца лента 2х сл 1шт
63181,ЛЕНТА,Б/полотенца ЛЕНТА 2х сл 1шт,2.0,79.99,159.98,лента,б/полотенца лента 2х сл 1шт
63304,ЛЕНТА,Б/полотенца ЛЕНТА 2х сл 1шт,1.0,79.99,79.99,лента,б/полотенца лента 2х сл 1шт


In [245]:
real[real['lower_name'].str.find('кп/') != -1]

,shop_name,name,quantity,price,sum,name_new,lower_name
41040,АТАК,КП/ЯЙЦО ДЕРЕВЕНСКОЕ,1.0,73.90,73.90,деревенское,кп/яйцо деревенское
41041,АТАК,КП/ЯЙЦО ДЕРЕВЕНСКОЕ,1.0,73.90,73.90,деревенское,кп/яйцо деревенское
41046,АТАК,КП/МАСЛО СЛИВОЧНОЕ,1.0,125.00,125.00,сливочное,кп/масло сливочное
41129,АТАК,КП/ТВОРОГ 5% 300Г,1.0,74.10,74.10,,кп/творог 5% 300г
41227,АТАК,КП/СТАКАН ПЛОМБИР,1.0,19.99,19.99,пломбир,кп/стакан пломбир
...,...,...,...,...,...,...,...
47330,АШАН,КП/ТЕТРАДЬ 60Л КЛЕТКА НА СПИР.,1.0,31.00,31.00,клетка спир,кп/тетрадь 60л клетка на спир.
47621,АШАН,КП/РИС КРУГЛОЗЕР.800,1.0,69.99,69.99,круглозер,кп/рис круглозер.800
47667,АШАН,"КП/МОЛОКО 3,4-6%930Г",1.0,57.00,57.00,,"кп/молоко 3,4-6%930г"
47807,АШАН,КП/САЛФ В БОКСЕ 60ШТ,2.0,53.99,107.98,боксе,кп/салф в боксе 60шт


In [ ]:
'йогурт/смусси': 'йогурт',
'т/бумага': 'туалетная бумага',
'deluxe/зеваделюк': 'deluxe',
'укроп/петрушк': 'зелень',
'nemoloko/немолоко': 'nemoloko',
'huggies/хаггис': 'huggies',
'figaro/фигаро': 'figaro',
'кп/спагетти': 'красная птица макароны',
'б/полотенца': 'бумажные полотенца'

In [ ]:
nemoloko/немолоко  huggies/хаггис  figaro/фигаро  говядина/св  кп/спагетти т/бумага  песок/сахар Б/полотенца

In [ ]:
 твор упак карта дск дом стол смесь цена короткоплодные скретч gold паст изд колб перекресток зеленый домик бумага йог мытая прод микс печ дней

In [ ]:
 твор упак карта дск дом стол смесь цена короткоплодные скретч gold паст изд колб перекресток зеленый домик бумага йог мытая прод микс печ дней

In [ ]:
class Namefinder:
    def __init__(self, df: pd.Series):
        self.df = df.copy()
        
    def is_one_word(name: str) -> bool:
        return len(name.split()) == 1
    
    def find_english_word(word):
        pass
    
    

In [23]:
t

0                                          желудки
1                                              сок
2                                  томат бакинский
3                                     йогурт вишня
4                                          антижир
5                                 кислота уксусная
6                                     сыр слим фит
7                                          стикеры
8                                 грибы шампиньоны
9                              агрикола аква декор
10                                          бананы
11                                         базилик
12                                           батон
13                                сливочное clever
14                                  хлеб ароматный
15                                  груша сезонная
16                                    балык свиной
17                         кофе nescafe gold раств
18                                   перец красный
19                             

In [24]:
tmp

,shop_name,Название,product,brand,category
0,АТАК,желудки 500г/подл/тр,желудки,троекурово,"Птица, мясо, деликатесы"
1,АТАК,"сок ябл/банан 6м 0,2",сок,-,"Воды, соки, напитки"
2,АТАК,томат бакинский вес,томат,-,"Овощи, фрукты, ягоды"
3,АТАК,йог вишня 250гр,йогурт,-,"Молоко, сыр, яйца"
4,АТАК,ср-во антижир 500мл,средство антижир,-,"Красота, гигиена, бытовая химия"
5,АШАН,кислота уксусная 70%,кислота уксусная,-,"Макароны, крупы, специи"
6,АШАН,сыр слим фит 27% 250,сыр,-,"Молоко, сыр, яйца"
7,АШАН,стикеры 7шт 17х55,стикеры,-,Другое
8,АШАН,грибы шампиньоны вес,грибы,-,"Овощи, фрукты, ягоды"
9,АШАН,агрикола аква декор,агрикола,-,Товары для дома и дачи


'   sdf'

In [35]:
def remove_all_words_which_contains_number(name: str):
    return ' '.join(re.sub(r'(\w*\d\w*)|(\w*/\w*)', '', word).strip() for word in name.split())

In [37]:
tmp.head()

,shop_name,Название,product,brand,category
0,АТАК,желудки 500г/подл/тр,желудки,троекурово,"Птица, мясо, деликатесы"
1,АТАК,"сок ябл/банан 6м 0,2",сок,-,"Воды, соки, напитки"
2,АТАК,томат бакинский вес,томат,-,"Овощи, фрукты, ягоды"
3,АТАК,йог вишня 250гр,йогурт,-,"Молоко, сыр, яйца"
4,АТАК,ср-во антижир 500мл,средство антижир,-,"Красота, гигиена, бытовая химия"


In [84]:
t = pd.read_excel('data/data.xlsx')

In [86]:
target_shops = [
    'АО "ТОРГОВЫЙ ДОМ "ПЕРЕКРЕСТОК"', 'ООО "АГРОТОРГ"', 'ООО "ЛЕНТА"',
   'АО "ДИКСИ ЮГ"', 'ООО "АШАН"', 'АО "ТАНДЕР"', 'ООО "БИЛЛА"', 'ООО "О`КЕЙ"',
    'ООО "АГРОАСПЕКТ"', 'ООО "АТАК"'
]

shops_rename = {
    'АО "ТОРГОВЫЙ ДОМ "ПЕРЕКРЕСТОК"': 'ПЕРЕКРЕСТОК',
    'ООО "АГРОТОРГ"': 'ПЯТЕРОЧКА',
    'ООО "ЛЕНТА"': 'ЛЕНТА',
    'АО "ДИКСИ ЮГ"': 'ДИКСИ',
    'ООО "АШАН"': 'АШАН',
    'АО "ТАНДЕР"': 'МАГНИТ',
    'ООО "БИЛЛА"': 'БИЛЛА',
    'ООО "О`КЕЙ"': 'О`КЕЙ',
    'ООО "АГРОАСПЕКТ"': 'ПЯТЕРОЧКА',
    'ООО "АТАК"': 'АТАК'
}

t = t[t['shop_name'].isin(target_shops)].copy()
t['shop_name'] = t['shop_name'].apply(lambda x: shops_rename[x])

In [88]:
t['name'] = t['name'].str.lower()

In [90]:
from collections import Counter

In [131]:
not '400г'.isalpha()

True

# Counter

In [25]:
from collections import Counter

C = Counter()
def lol(name):
    global C
    for word in name.split():
        if not word.isalpha():
            continue
        C[word] += 1
        
real['name'].apply(lol);
C.most_common(20)

In [110]:
t['name'][t['name'].str.find('пакет') >= 0].sample(10)

21475          2:3305976 пакет перекресток майка
16858          5:3305976 пакет перекресток майка
59955    пакеты д/мус фрекен бок 35л, с руч 30шт
21298          3:3305976 пакет перекресток майка
31337            3300573 пакет пятерочка 65х40см
51114                          пакет билла больш
4563                  пакет-майка дикси пнд 12кг
50912                        пакет билла био мал
9049                          пакет-майка магнит
61143      тушка ц/б кудашка 1сорт пакет охл вес
Name: name, dtype: object

Counter({'минтай филе б/к св/мор п/пак 8': 1,
         'бзмж молоко страна васильки у/': 7,
         'сушки тараллини с чесноком 180': 3,
         'яйцо куриное столовое 1категор': 19,
         'печенье юбилейное витаминизиро': 7,
         'лук репчатый вес': 41,
         'масло подсолнечное раф 0,9л': 5,
         'бзмж молоко сгущ. волоконовско': 1,
         'майонез ряба оливковый 67% д/п': 2,
         'огурцы 600г': 6,
         'яйцо куриное столовое отборное': 17,
         'крупа гречневая макфа ядрица 8': 2,
         'майонез московск.провансаль кл': 4,
         'соль поваренная пищ.экстра/мел': 5,
         'кетчуп heinz томатный д/пак 35': 8,
         'перец красный вес': 36,
         'бзмж морожен.русский холод пло': 2,
         'томаты вес': 25,
         'игрушка машинка джип 12х9 см а': 1,
         'бзмж молоко сгущенное с/сах ро': 9,
         'чипсы lay`s/лейс из печи в асс': 1,
         'салфетки бумажные yes`s/ес кос': 2,
         'капуста белокочанная ранняя ве': 16,
     